In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import re
#import data_helpers
#from text_cnn import TextCNN
from tensorflow.contrib import learn

In [2]:
def listtext(path):
    list_name = []
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            listdir(file_path, list_name)
        else:
            list_name.append(file_path)
    list_text = []
    for files in list_name:
        file = open(files,'r')
        list_text.append(file.readlines()[0])
    return list_text

In [3]:
os.getcwd()

'/mnt/md1/a503wangsiqi'

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

In [5]:
current_path = os.getcwd()

path_neg = current_path + '/acllmdb/aclImdb/train/neg'
path_pos = current_path + '/acllmdb/aclImdb/train/pos'
#path_unsup = current_path + 'F:/acllmdb/aclImdb/train/unsup'

pos_reviews = listtext(path_pos)
neg_reviews = listtext(path_neg)
#unsup_reviews = listtext(path_unsup)

In [6]:
tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
#tf.flags.DEFINE_string("positive_data_file", path_pos, "Data source for the positive data.")
#tf.flags.DEFINE_string("negative_data_file", path_neg, "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", True, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS

In [7]:
def preprocess():
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    x_text, y = load_data_and_labels(pos_reviews, neg_reviews)

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev

def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with tf.device('/cpu:0'), sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=FLAGS.embedding_dim,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.l2_reg_lambda)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Write vocabulary
            vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                
                #with tf.device('/cpu:0'):
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
            #batches_dev = data_helpers.batch_iter(
            #    list(zip(x_dev, y_dev)), FLAGS.batch_size, FLAGS.num_epochs)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))

In [8]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout 输入输出以及保留率
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional) 保存l2的正则损失
        l2_loss = tf.constant(0.0)

        # Embedding layer 初始化词向量
        #with tf.device('/cpu:0'), tf.name_scope("embedding"):
        with tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)#？

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):#卷积层的定义 卷积核大小
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                #卷积核的四个维度分别是卷积单词个数 词向量的维度  输入层的维度 输出的卷积维度
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")#卷积核的初始化
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],#把卷积之后的多维度向量 池化到一个维度
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)#将全连接层加入l2正则化
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [9]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels(pos_reviews, neg_reviews):
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    # Load data from files
    '''
    positive_examples = list(open(positive_data_file, "r").readlines())
    positive_examples = [s.strip() for s in positive_examples]
    negative_examples = list(open(negative_data_file, "r").readlines())
    negative_examples = [s.strip() for s in negative_examples]

    # Split by words
    x_text = positive_examples + negative_examples
    '''
    x_text = pos_reviews + neg_reviews
    x_text = [clean_str(sent) for sent in x_text]
    # Generate labels
    positive_labels = [[0, 1] for _ in pos_reviews]
    negative_labels = [[1, 0] for _ in neg_reviews]
    y = np.concatenate([positive_labels, negative_labels], 0)
    return [x_text, y]


def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]


In [10]:
x_train, y_train, vocab_processor, x_dev, y_dev = preprocess()

Loading data...
Vocabulary Size: 80918
Train/Dev split: 22500/2500


In [ ]:
train(x_train, y_train, vocab_processor, x_dev, y_dev)

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

2018-07-12T17:48:39.365839: step 99, loss 1.45037, acc 0.671875
2018-07-12T17:48:40.976723: step 100, loss 1.28639, acc 0.609375

Evaluation:
2018-07-12T17:48:58.886172: step 100, loss 0.687164, acc 0.6244

Saved model checkpoint to /mnt/md1/a503wangsiqi/runs/1531388756/checkpoints/model-100

2018-07-12T17:49:01.311526: step 101, loss 1.45864, acc 0.546875
2018-07-12T17:49:02.870113: step 102, loss 2.05803, acc 0.546875
2018-07-12T17:49:04.453861: step 103, loss 2.6602, acc 0.453125
2018-07-12T17:49:06.108705: step 104, loss 1.49275, acc 0.625
2018-07-12T17:49:07.683405: step 105, loss 1.50004, acc 0.59375
2018-07-12T17:49:09.251419: step 106, loss 1.57666, acc 0.53125
2018-07-12T17:49:10.799875: step 107, loss 1.77096, acc 0.5
2018-07-12T17:49:12.413569: step 108, loss 1.74562, acc 0.59375
2018-07-12T17:49:14.039733: step 109, loss 2.69513, acc 0.375
2018-07-12T17:49:15.586603: step 110, loss 1.99697, acc 0.5625
2018-07-12T17:49:17.138935: step 111, loss 1.59183, acc 0.609375
2018-07-

2018-07-12T17:52:33.610796: step 222, loss 1.16518, acc 0.578125
2018-07-12T17:52:35.231420: step 223, loss 1.21825, acc 0.46875
2018-07-12T17:52:36.821966: step 224, loss 1.13007, acc 0.59375
2018-07-12T17:52:38.375180: step 225, loss 0.93358, acc 0.625
2018-07-12T17:52:39.963701: step 226, loss 1.27526, acc 0.59375
2018-07-12T17:52:41.592602: step 227, loss 1.14842, acc 0.578125
2018-07-12T17:52:43.266977: step 228, loss 1.18726, acc 0.5625
2018-07-12T17:52:44.892515: step 229, loss 1.33516, acc 0.546875
2018-07-12T17:52:46.486666: step 230, loss 1.03039, acc 0.625
2018-07-12T17:52:48.047188: step 231, loss 1.29622, acc 0.59375
2018-07-12T17:52:49.652710: step 232, loss 1.49041, acc 0.53125
2018-07-12T17:52:51.238569: step 233, loss 1.19465, acc 0.59375
2018-07-12T17:52:52.786497: step 234, loss 0.9275, acc 0.671875
2018-07-12T17:52:54.329133: step 235, loss 1.03828, acc 0.6875
2018-07-12T17:52:55.952853: step 236, loss 1.2379, acc 0.5625
2018-07-12T17:52:57.554345: step 237, loss 0.

2018-07-12T17:56:14.735724: step 347, loss 0.684853, acc 0.671875
2018-07-12T17:56:16.419611: step 348, loss 0.904254, acc 0.578125
2018-07-12T17:56:18.064365: step 349, loss 0.624791, acc 0.65625
2018-07-12T17:56:19.623259: step 350, loss 0.70719, acc 0.65625
2018-07-12T17:56:21.212027: step 351, loss 0.727171, acc 0.71875
2018-07-12T17:56:22.210823: step 352, loss 0.793633, acc 0.638889
2018-07-12T17:56:23.775781: step 353, loss 0.741915, acc 0.59375
2018-07-12T17:56:25.340623: step 354, loss 0.792622, acc 0.609375
2018-07-12T17:56:26.922775: step 355, loss 0.784125, acc 0.59375
2018-07-12T17:56:28.547276: step 356, loss 0.765951, acc 0.59375
2018-07-12T17:56:30.221228: step 357, loss 0.782151, acc 0.625
2018-07-12T17:56:31.796198: step 358, loss 0.649875, acc 0.65625
2018-07-12T17:56:33.408385: step 359, loss 0.745584, acc 0.625
2018-07-12T17:56:35.063988: step 360, loss 0.787494, acc 0.65625
2018-07-12T17:56:36.716089: step 361, loss 0.875559, acc 0.59375
2018-07-12T17:56:38.311898

2018-07-12T17:59:50.320997: step 471, loss 0.665257, acc 0.625
2018-07-12T17:59:51.898992: step 472, loss 0.681732, acc 0.640625
2018-07-12T17:59:53.494179: step 473, loss 0.622925, acc 0.734375
2018-07-12T17:59:55.085303: step 474, loss 0.64529, acc 0.578125
2018-07-12T17:59:56.711304: step 475, loss 0.573416, acc 0.71875
2018-07-12T17:59:58.312349: step 476, loss 0.603266, acc 0.6875
2018-07-12T18:00:00.019616: step 477, loss 0.577599, acc 0.75
2018-07-12T18:00:01.679119: step 478, loss 0.537196, acc 0.734375
2018-07-12T18:00:03.302616: step 479, loss 0.581242, acc 0.703125
2018-07-12T18:00:04.905492: step 480, loss 0.679767, acc 0.6875
2018-07-12T18:00:06.494511: step 481, loss 0.568599, acc 0.71875
2018-07-12T18:00:08.078277: step 482, loss 0.599387, acc 0.71875
2018-07-12T18:00:09.626868: step 483, loss 0.734679, acc 0.65625
2018-07-12T18:00:11.179191: step 484, loss 0.607705, acc 0.703125
2018-07-12T18:00:12.774258: step 485, loss 0.489923, acc 0.78125
2018-07-12T18:00:14.409314:

...

2018-07-12T19:28:34.321385: step 3397, loss 0.122199, acc 0.9375
2018-07-12T19:28:36.013691: step 3398, loss 0.1788, acc 0.9375
2018-07-12T19:28:38.006197: step 3399, loss 0.246865, acc 0.921875
2018-07-12T19:28:39.783561: step 3400, loss 0.184068, acc 0.953125

Evaluation:
2018-07-12T19:28:59.029812: step 3400, loss 0.299358, acc 0.8788

Saved model checkpoint to /mnt/md1/a503wangsiqi/runs/1531388756/checkpoints/model-3400

2018-07-12T19:29:01.484022: step 3401, loss 0.165179, acc 0.9375
2018-07-12T19:29:03.113235: step 3402, loss 0.285716, acc 0.90625
2018-07-12T19:29:04.914419: step 3403, loss 0.127706, acc 0.953125
2018-07-12T19:29:06.565870: step 3404, loss 0.0826354, acc 0.984375
2018-07-12T19:29:08.308818: step 3405, loss 0.17037, acc 0.921875
2018-07-12T19:29:09.895512: step 3406, loss 0.121743, acc 0.96875
2018-07-12T19:29:11.474204: step 3407, loss 0.285164, acc 0.890625
2018-07-12T19:29:13.093873: step 3408, loss 0.176501, acc 0.9375
2018-07-12T19:29:14.721587: step 3409, lo

2018-07-12T19:32:30.917283: step 3516, loss 0.260262, acc 0.90625
2018-07-12T19:32:32.613115: step 3517, loss 0.131237, acc 0.953125
2018-07-12T19:32:34.278953: step 3518, loss 0.17721, acc 0.9375
2018-07-12T19:32:35.881477: step 3519, loss 0.118465, acc 0.96875
2018-07-12T19:32:36.898921: step 3520, loss 0.230175, acc 0.916667
2018-07-12T19:32:38.529467: step 3521, loss 0.153555, acc 0.90625
2018-07-12T19:32:40.112433: step 3522, loss 0.0467327, acc 0.96875
2018-07-12T19:32:41.736316: step 3523, loss 0.0816384, acc 0.984375
2018-07-12T19:32:43.401055: step 3524, loss 0.253243, acc 0.90625
2018-07-12T19:32:45.046028: step 3525, loss 0.13338, acc 0.953125
2018-07-12T19:32:46.686044: step 3526, loss 0.0848224, acc 0.984375
2018-07-12T19:32:48.260419: step 3527, loss 0.153114, acc 0.921875
2018-07-12T19:32:49.907479: step 3528, loss 0.110779, acc 0.9375
2018-07-12T19:32:51.557573: step 3529, loss 0.0774974, acc 0.96875
2018-07-12T19:32:53.216436: step 3530, loss 0.0567385, acc 0.984375
20

2018-07-12T19:36:10.259875: step 3637, loss 0.0502535, acc 0.984375
2018-07-12T19:36:11.892845: step 3638, loss 0.0681184, acc 0.984375
2018-07-12T19:36:13.557003: step 3639, loss 0.0815325, acc 0.96875
2018-07-12T19:36:15.197259: step 3640, loss 0.0630341, acc 0.984375
2018-07-12T19:36:16.848680: step 3641, loss 0.170832, acc 0.9375
2018-07-12T19:36:18.518794: step 3642, loss 0.0759196, acc 0.96875
2018-07-12T19:36:20.149524: step 3643, loss 0.286317, acc 0.890625
2018-07-12T19:36:21.760390: step 3644, loss 0.116982, acc 0.9375
2018-07-12T19:36:23.432614: step 3645, loss 0.0716173, acc 0.96875
2018-07-12T19:36:25.067154: step 3646, loss 0.0978018, acc 0.9375
2018-07-12T19:36:26.747841: step 3647, loss 0.0697633, acc 0.953125
2018-07-12T19:36:28.406319: step 3648, loss 0.0327857, acc 0.984375
2018-07-12T19:36:30.017801: step 3649, loss 0.199724, acc 0.953125
2018-07-12T19:36:31.644575: step 3650, loss 0.199603, acc 0.890625
2018-07-12T19:36:33.260538: step 3651, loss 0.0793399, acc 0.9

2018-07-12T19:39:49.819293: step 3759, loss 0.134837, acc 0.9375
2018-07-12T19:39:51.467737: step 3760, loss 0.139094, acc 0.9375
2018-07-12T19:39:53.062644: step 3761, loss 0.0923502, acc 0.984375
2018-07-12T19:39:54.660871: step 3762, loss 0.111937, acc 0.953125
2018-07-12T19:39:56.279126: step 3763, loss 0.0820715, acc 0.96875
2018-07-12T19:39:58.006661: step 3764, loss 0.0660487, acc 0.96875
2018-07-12T19:39:59.648148: step 3765, loss 0.107239, acc 0.953125
2018-07-12T19:40:01.256358: step 3766, loss 0.0884236, acc 0.96875
2018-07-12T19:40:02.964441: step 3767, loss 0.143395, acc 0.9375
2018-07-12T19:40:04.603016: step 3768, loss 0.120961, acc 0.953125
2018-07-12T19:40:06.312503: step 3769, loss 0.10411, acc 0.96875
2018-07-12T19:40:08.044799: step 3770, loss 0.163866, acc 0.9375
2018-07-12T19:40:09.770473: step 3771, loss 0.0775496, acc 0.953125
2018-07-12T19:40:11.411217: step 3772, loss 0.089556, acc 0.96875
2018-07-12T19:40:13.109960: step 3773, loss 0.11958, acc 0.9375
2018-07

2018-07-12T19:43:28.139328: step 3880, loss 0.0582735, acc 0.96875
2018-07-12T19:43:29.766966: step 3881, loss 0.036242, acc 1
2018-07-12T19:43:31.378733: step 3882, loss 0.0866558, acc 0.953125
2018-07-12T19:43:32.968752: step 3883, loss 0.0829065, acc 0.953125
2018-07-12T19:43:34.586761: step 3884, loss 0.0771352, acc 0.96875
2018-07-12T19:43:36.197991: step 3885, loss 0.0673631, acc 0.96875
2018-07-12T19:43:37.847493: step 3886, loss 0.0371459, acc 1
2018-07-12T19:43:39.532902: step 3887, loss 0.0654754, acc 0.96875
2018-07-12T19:43:41.196504: step 3888, loss 0.090727, acc 0.96875
2018-07-12T19:43:42.834148: step 3889, loss 0.0980639, acc 0.953125
2018-07-12T19:43:44.487264: step 3890, loss 0.102703, acc 0.953125
2018-07-12T19:43:46.108199: step 3891, loss 0.0332321, acc 1
2018-07-12T19:43:47.744412: step 3892, loss 0.091625, acc 0.96875
2018-07-12T19:43:49.399765: step 3893, loss 0.031875, acc 1
2018-07-12T19:43:51.055466: step 3894, loss 0.0558781, acc 0.984375
2018-07-12T19:43:52

In [ ]:
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Eval Parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_string("checkpoint_dir", "", "Checkpoint directory from training run")
tf.flags.DEFINE_boolean("eval_train", False, "Evaluate on all training data")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")


FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

# CHANGE THIS: Load data. Load your own data here
if FLAGS.eval_train:
    #x_raw, y_test = load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)
    x_raw, y_test = load_data_and_labels(pos_reviews, neg_reviews)
    y_test = np.argmax(y_test, axis=1)
else:
    x_raw = ["a masterpiece four years in the making", "everything is off."]
    y_test = [1, 0]

# Map data into vocabulary
vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

print("\nEvaluating...\n")

# Evaluation
# ==================================================
checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy if y_test is defined
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

# Save the evaluation to a csv
predictions_human_readable = np.column_stack((np.array(x_raw), all_predictions))
out_path = os.path.join(FLAGS.checkpoint_dir, "..", "prediction.csv")
print("Saving evaluation to {0}".format(out_path))
with open(out_path, 'w') as f:
    csv.writer(f).writerows(predictions_human_readable)